In [1]:
import pandas as pd
import sqlite3
from sqlite3 import Error
import os
import datetime as dt

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print('Error in conn', e)
    return conn


def create_table(conn, create_table_sql_cmd):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql_cmd)
    except Error as e:
        print('Error in create table', e)
    return


def insert_values_in_table_users(conn, task):
    sql_query = ''' INSERT INTO users (name, date_joined) VALUES (?, ?) '''
    cur = conn.cursor()
    cur.execute(sql_query, task)
    conn.commit()
    print('inserted -> ', task)

    
def insert_values_in_table_purchases(conn, task):
    sql_query = ''' INSERT INTO purchases (user, date_purchased, item, price) VALUES (?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql_query, task)
    conn.commit()
    print('inserted -> ', task)

In [2]:
db = "assignment_db.db"

# remove existing DB file to avoid rows duplication
if os.path.exists(db):
    os.remove(db)

# create a sqlite db connection
conn = create_connection(db)

if conn is None:
    print(f"Error, cannot create db connection to {db}")
    raise SystemExit

# Part 1

In [3]:
create_users_table = """ CREATE TABLE IF NOT EXISTS users (
                            name TEXT NOT NULL,
                            date_joined TEXT
                        ); """

create_purchases_table = """ CREATE TABLE IF NOT EXISTS purchases (
                                user TEXT NOT NULL,
                                date_purchased TEXT NOT NULL,
                                item TEXT NOT NULL,
                                price REAL NOT NULL
                            ); """

# create tables
create_table(conn, create_users_table)
create_table(conn, create_purchases_table)

#############################################################

# insert data in both tables
users_values = [('John', '2011-01-01'),
                ('Dave', '2009-04-02'),
                ('Mary', '2008-03-04')]

for val in users_values:
    insert_values_in_table_users(conn, val)

purchase_values = [('John', '2011-02-04', 'SK2341', 34.54),
                   ('John', '2012-01-04', 'LS2414', 94.98),
                   ('John', '2012-01-05', 'LS2414', 1),
                   ('Dave', '2021-09-06', 'LS7734', 5),
                   ('Dave', '2021-07-06', 'LS7734', 45),
                   ]

for val in purchase_values:
    insert_values_in_table_purchases(conn, val)

#############################################################

inserted ->  ('John', '2011-01-01')
inserted ->  ('Dave', '2009-04-02')
inserted ->  ('Mary', '2008-03-04')
inserted ->  ('John', '2011-02-04', 'SK2341', 34.54)
inserted ->  ('John', '2012-01-04', 'LS2414', 94.98)
inserted ->  ('John', '2012-01-05', 'LS2414', 1)
inserted ->  ('Dave', '2021-09-06', 'LS7734', 5)
inserted ->  ('Dave', '2021-07-06', 'LS7734', 45)


## Display tables

In [4]:
table_users = pd.read_sql_query('SELECT * FROM users', conn)
table_users

,name,date_joined
0,John,2011-01-01
1,Dave,2009-04-02
2,Mary,2008-03-04


In [5]:
table_purchases = pd.read_sql_query('SELECT * FROM purchases', conn)
table_purchases

,user,date_purchased,item,price
0,John,2011-02-04,SK2341,34.54
1,John,2012-01-04,LS2414,94.98
2,John,2012-01-05,LS2414,1.00
3,Dave,2021-09-06,LS7734,5.00
4,Dave,2021-07-06,LS7734,45.00


### Q1.1) What are the total expenditures by the user (all time)? 

In [6]:
total_exp_user_all_time_query = ''' SELECT A.name, SUM(B.price) as user_purchase_total
                                    FROM users as A LEFT JOIN purchases as B 
                                    ON A.name = B.user
                                    GROUP BY B.user;
                                '''
# WHERE users.name = purchases.user
pd.read_sql_query(total_exp_user_all_time_query, conn)

,name,user_purchase_total
0,Mary,NaN
1,Dave,50.00
2,John,130.52


### Q1.2) What is the total expenditure by item?

In [7]:
total_exp_user_by_item_query = '''  SELECT purchases.item, SUM(purchases.price) as purchase_total
                                    FROM purchases
                                    GROUP BY purchases.item  '''

pd.read_sql_query(total_exp_user_by_item_query, conn)

,item,purchase_total
0,LS2414,95.98
1,LS7734,50.00
2,SK2341,34.54


### Q2) What fraction of customers have never made a purchase?

In [8]:
frac_never_purchased_query = ''' SELECT 1 - ROUND((SELECT COUNT(DISTINCT p.user) FROM purchases as p) / CAST(COUNT(u.name) as REAL) , 2) 
                                 as frac_users_not_purchased FROM users as u; '''

pd.read_sql_query(frac_never_purchased_query, conn)

,frac_users_not_purchased
0,0.33


### Q3) Can we make a table based on purchases that has a column “bought_before” that is a boolean, with a value of 1 if they have purchased that item before and a value of 0 if it’s their first time buying that item?

In [9]:
table_purchases

,user,date_purchased,item,price
0,John,2011-02-04,SK2341,34.54
1,John,2012-01-04,LS2414,94.98
2,John,2012-01-05,LS2414,1.00
3,Dave,2021-09-06,LS7734,5.00
4,Dave,2021-07-06,LS7734,45.00


In [10]:
pd.read_sql_query('''
    SELECT user, date_purchased, item, price, CASE WHEN rnk = 1
                                                   THEN 0
                                                   ELSE 1
                                              END as bought_before
    FROM (
            SELECT *, DENSE_RANK() OVER (
                        PARTITION BY user, item
                        ORDER BY date_purchased ASC
                    ) as rnk
            FROM purchases
        ) as A;
''', conn)

,user,date_purchased,item,price,bought_before
0,Dave,2021-07-06,LS7734,45.00,0
1,Dave,2021-09-06,LS7734,5.00,1
2,John,2012-01-04,LS2414,94.98,0
3,John,2012-01-05,LS2414,1.00,1
4,John,2011-02-04,SK2341,34.54,0


### ------------------------------------------------------------------------------------

## Rough work to test logic

In [11]:
# More Assumptions based logic
# USing Pandas Df logic (Assumption 1)
# Checking from items in purchases table, which all users have bought those items (1) and which have not (0)

table_bought_before = pd.DataFrame(columns=['name', 'item', 'bought_before'])

for name in table_users.name.unique():
    df_unique = table_purchases[table_purchases['user']==name].item.unique()
    for item in table_purchases.item.unique():
        if item in df_unique:
            print(f'"{name}" has bought "{item}" before')
            table_bought_before.loc[len(table_bought_before)] = [name, item, 1]
        else:
            print(f'"{name}" is purchasing "{item}" first time')
            table_bought_before.loc[len(table_bought_before)] = [name, item, 0]

table_bought_before

"John" has bought "SK2341" before
"John" has bought "LS2414" before
"John" is purchasing "LS7734" first time
"Dave" is purchasing "SK2341" first time
"Dave" is purchasing "LS2414" first time
"Dave" has bought "LS7734" before
"Mary" is purchasing "SK2341" first time
"Mary" is purchasing "LS2414" first time
"Mary" is purchasing "LS7734" first time


,name,item,bought_before
0,John,SK2341,1
1,John,LS2414,1
2,John,LS7734,0
3,Dave,SK2341,0
4,Dave,LS2414,0
5,Dave,LS7734,1
6,Mary,SK2341,0
7,Mary,LS2414,0
8,Mary,LS7734,0


In [12]:
# USing Pandas Df logic (Assumption 2)
# separetly check for item and user

table_bought_before = pd.DataFrame(columns=['name', 'item', 'bought_before'])

item = 'LS2414'
name = 'John'

if name in table_purchases.user.unique() and item in table_purchases.item.unique():
    print(f'"{name}" has bought "{item}" before')
    table_bought_before.loc[len(table_bought_before)] = [name, item, 1]
else:
    print(f'"{name}" is purchasing "{item}" first time')
    table_bought_before.loc[len(table_bought_before)] = [name, item, 0]

table_bought_before.drop_duplicates(inplace=True)

table_bought_before

"John" has bought "LS2414" before


,name,item,bought_before
0,John,LS2414,1


In [15]:
# pd.read_sql_query(''' With cte as (SELECT DISTINCT u.name, p.item
#                       FROM purchases as p
#                       CROSS JOIN users as u ORDER BY u.name)
                      
#                       SELECT c.name, c.item,  CASE WHEN (EXISTS (SELECT 1 FROM purchases as p WHERE p.user=c.name)) AND (EXISTS (SELECT 1 FROM purchases as p WHERE p.item=c.item)) 
#                                                    THEN 1
#                                                    ELSE 0
#                                           END as bought_before
                      
#                       FROM cte as c
#                   ''' , conn)

# pd.read_sql_query("SELECT EXISTS(SELECT item FROM purchases) as bought_before;", conn)

# pd.read_sql_query('''SELECT DISTINCT item FROM purchases;''', conn)

# WHERE EXISTS (SELECT item FROM purchases)

# pd.read_sql_query('SELECT item, IF(count(item)>1, 1, 0) FROM purchases;', conn)

### ------------------------------------------------------------------------------------

### Q4) How would we tell if there’s a seasonal pattern for certain items? What queries might help us understand any seasonal patterns that exist?

In [16]:
# Approach:
# find qyarter from date 
# group by quarte, item
# and count the users

# Answer would be - Which ever qurater has more items bought

q = '''
        SELECT *, count(item) as num_of_times_item_bought
        
        FROM (SELECT *, CASE 
                            WHEN CAST(strftime('%m', date_purchased) as INTEGER) BETWEEN 1 AND 3 THEN "Q1"
                            WHEN CAST(strftime('%m', date_purchased) as INTEGER) BETWEEN 4 AND 6 THEN "Q2"
                            WHEN CAST(strftime('%m', date_purchased) as INTEGER) BETWEEN 7 AND 9 THEN "Q3"
                            ELSE "Q4"
                         END AS quarter
              FROM purchases)
         
         GROUP BY quarter, item; 
     '''

pd.read_sql_query(q, conn)

,user,date_purchased,item,price,quarter,num_of_times_item_bought
0,John,2012-01-04,LS2414,94.98,Q1,2
1,John,2011-02-04,SK2341,34.54,Q1,1
2,Dave,2021-09-06,LS7734,5.00,Q3,2


# Part 2

In [17]:
def insert_values_in_table_sessions(conn, task):
    sql_query = ''' INSERT INTO sessions (session_id, customer_id, started_at, ended_at, utm_source, utm_medium, utm_campaign) VALUES (?, ?, ?, ?, ?, ?, ?); '''
    cur = conn.cursor()
    cur.execute(sql_query, task)
    conn.commit()
    print('inserted -> ', task)


def insert_values_in_table_conversions(conn, task):
    sql_query = ''' INSERT INTO conversions (customer_id, converted_at) VALUES (?, ?); '''
    cur = conn.cursor()
    cur.execute(sql_query, task)
    conn.commit()
    print('inserted -> ', task)

    
create_sessions_table = """ CREATE TABLE IF NOT EXISTS sessions (
                            session_id INT NOT NULL,
                            customer_id INT NOT NULL,
                            started_at TEXT,
                            ended_at TEXT,
                            utm_source TEXT,
                            utm_medium TEXT,
                            utm_campaign TEXT
                        ); """

create_conversions_table = """ CREATE TABLE IF NOT EXISTS conversions (
                                customer_id INT NOT NULL,
                                converted_at TEXT NOT NULL
                            ); """

# create tables
create_table(conn, create_sessions_table)
create_table(conn, create_conversions_table)

#############################################################

# insert data in both tables
session_values = [(1, 2956, '2020-02-01 12:55:16', '2020-02-01 12:55:47', 'facebook_ads', 'paid_social', 'xyz'),
                  (2, 4, '2020-02-02 13:06:47', '2020-02-02 13:18:56', 'facebook_ads', 'paid_social', 'freeshipping'),
                  (3, 1170, '2020-02-03 12:15:00', '2020-02-03 12:15:19', 'youtube1', 'social_video', 'inf1'),
                  (4, 1170, '2020-02-03 17:15:00', '2020-02-03 17:15:19', 'facebook_ads', 'paid_social', 'inf1')
                 ]

for val in session_values:
    insert_values_in_table_sessions(conn, val)

conversion_values = [(1170, '2020-02-03 14:20:08'),
                     (2014, '2020-02-04 4:30:21'),
                     (2265, '2020-02-04 9:43:35')]

for val in conversion_values:
    insert_values_in_table_conversions(conn, val)

#############################################################

inserted ->  (1, 2956, '2020-02-01 12:55:16', '2020-02-01 12:55:47', 'facebook_ads', 'paid_social', 'xyz')
inserted ->  (2, 4, '2020-02-02 13:06:47', '2020-02-02 13:18:56', 'facebook_ads', 'paid_social', 'freeshipping')
inserted ->  (3, 1170, '2020-02-03 12:15:00', '2020-02-03 12:15:19', 'youtube1', 'social_video', 'inf1')
inserted ->  (4, 1170, '2020-02-03 17:15:00', '2020-02-03 17:15:19', 'facebook_ads', 'paid_social', 'inf1')
inserted ->  (1170, '2020-02-03 14:20:08')
inserted ->  (2014, '2020-02-04 4:30:21')
inserted ->  (2265, '2020-02-04 9:43:35')


### Additonal data added for answering questions

In [18]:
session_values = [(5, 2956, '2020-02-03 11:55:16', '2020-02-03 11:55:47', 'facebook_ads', 'paid_social', 'xyz'),
                  (6, 2956, '2020-02-03 10:55:16', '2020-02-03 10:55:47', 'facebook_ads', 'paid_social', 'xyz'),
                  (7, 2956, '2020-02-03 17:55:16', '2020-02-03 17:55:47', 'facebook_ads', 'paid_social', 'xyz'),
                  
                  (8, 1170, '2020-02-03 11:15:00', '2020-02-03 11:15:19', 'youtube1', 'social_video', 'inf1'),
                  (9, 1170, '2020-02-03 09:15:00', '2020-02-03 09:15:19', 'facebook_ads', 'paid_social', 'inf1')
                 ]

for val in session_values:
    insert_values_in_table_sessions(conn, val)

conversion_values = [(2956, '2020-02-03 14:20:08')]

for val in conversion_values:
    insert_values_in_table_conversions(conn, val)

inserted ->  (5, 2956, '2020-02-03 11:55:16', '2020-02-03 11:55:47', 'facebook_ads', 'paid_social', 'xyz')
inserted ->  (6, 2956, '2020-02-03 10:55:16', '2020-02-03 10:55:47', 'facebook_ads', 'paid_social', 'xyz')
inserted ->  (7, 2956, '2020-02-03 17:55:16', '2020-02-03 17:55:47', 'facebook_ads', 'paid_social', 'xyz')
inserted ->  (8, 1170, '2020-02-03 11:15:00', '2020-02-03 11:15:19', 'youtube1', 'social_video', 'inf1')
inserted ->  (9, 1170, '2020-02-03 09:15:00', '2020-02-03 09:15:19', 'facebook_ads', 'paid_social', 'inf1')
inserted ->  (2956, '2020-02-03 14:20:08')


## Display tables

In [19]:
table_sessions = pd.read_sql_query('SELECT * FROM sessions', conn)
table_sessions

,session_id,customer_id,started_at,ended_at,utm_source,utm_medium,utm_campaign
0,1,2956,2020-02-01 12:55:16,2020-02-01 12:55:47,facebook_ads,paid_social,xyz
1,2,4,2020-02-02 13:06:47,2020-02-02 13:18:56,facebook_ads,paid_social,freeshipping
2,3,1170,2020-02-03 12:15:00,2020-02-03 12:15:19,youtube1,social_video,inf1
3,4,1170,2020-02-03 17:15:00,2020-02-03 17:15:19,facebook_ads,paid_social,inf1
4,5,2956,2020-02-03 11:55:16,2020-02-03 11:55:47,facebook_ads,paid_social,xyz
5,6,2956,2020-02-03 10:55:16,2020-02-03 10:55:47,facebook_ads,paid_social,xyz
6,7,2956,2020-02-03 17:55:16,2020-02-03 17:55:47,facebook_ads,paid_social,xyz
7,8,1170,2020-02-03 11:15:00,2020-02-03 11:15:19,youtube1,social_video,inf1
8,9,1170,2020-02-03 09:15:00,2020-02-03 09:15:19,facebook_ads,paid_social,inf1


In [20]:
table_conversions = pd.read_sql_query('SELECT * FROM conversions', conn)
table_conversions

,customer_id,converted_at
0,1170,2020-02-03 14:20:08
1,2014,2020-02-04 4:30:21
2,2265,2020-02-04 9:43:35
3,2956,2020-02-03 14:20:08


In [21]:
# Imagine there are 10 marketing campaigns running by influencers and 10 paid ads campaigns running live. 

# You have 2 main source tables -> sessions and conversions , 
# you need to attribute each marketing campaign (by using UTM parameters) to

# 1) how many users it helped convert and 
# 2) how many touch points it takes a user to get converted

# Feel free to use any attribution modelling logic as possible and explain the logic, the code and
# the final output table you will arrive using these base raw tables to explain the same for the
# marketing team.

# Bonus if you can also add ad spend data & show ROI at the end, but not required.
# Just the logic/explanation will also do.

### Q1) how many users it helped convert

In [22]:
# we have to find conversions so, the customers whose 
# conversion time is after the ad starting time (JOIN tables to get this),
# will be considered and counting and grouping them
# by utm_source (my assumption - this is the source from where ads are being watched)
# will give us the num of users converted.

q = '''
        SELECT utm_source, COUNT(DISTINCT B.customer_id) as num_of_unique_users_converted 
        FROM sessions as A
        
        LEFT JOIN conversions as B 
        ON A.customer_id = B.customer_id AND A.started_at <= B.converted_at
        
        WHERE B.customer_id IS NOT NULL
        GROUP BY utm_source ;
'''

pd.read_sql_query(q, conn)

,utm_source,num_of_unique_users_converted
0,facebook_ads,2
1,youtube1,1


### Q2) how many touch points it takes a user to get converted

In [23]:
# intermediate table for understading the final result
pd.read_sql_query('''
                    SELECT *
                    FROM sessions as A

                    LEFT JOIN conversions as B
                    ON A.customer_id = B.customer_id AND A.started_at <= B.converted_at

                    WHERE B.customer_id IS NOT NULL;
                ''' , conn)

,session_id,customer_id,started_at,ended_at,utm_source,utm_medium,utm_campaign,customer_id,converted_at
0,3,1170,2020-02-03 12:15:00,2020-02-03 12:15:19,youtube1,social_video,inf1,1170,2020-02-03 14:20:08
1,8,1170,2020-02-03 11:15:00,2020-02-03 11:15:19,youtube1,social_video,inf1,1170,2020-02-03 14:20:08
2,9,1170,2020-02-03 09:15:00,2020-02-03 09:15:19,facebook_ads,paid_social,inf1,1170,2020-02-03 14:20:08
3,1,2956,2020-02-01 12:55:16,2020-02-01 12:55:47,facebook_ads,paid_social,xyz,2956,2020-02-03 14:20:08
4,5,2956,2020-02-03 11:55:16,2020-02-03 11:55:47,facebook_ads,paid_social,xyz,2956,2020-02-03 14:20:08
5,6,2956,2020-02-03 10:55:16,2020-02-03 10:55:47,facebook_ads,paid_social,xyz,2956,2020-02-03 14:20:08


In [24]:
# we have to find the touch points for the users that lead to their conversion,
# so first get the user and their conversion data (LEFT join like above intermediate table)
# and count the customer id by grouping them (since the conversion time is after the ad starting time)
# ASSUMPTION - all utm_source are equally influencing the customer to get converted.

q = '''
        SELECT B.customer_id, COUNT(A.customer_id) as num_of_touchpoints
        FROM sessions as A
        
        LEFT JOIN conversions as B 
        ON A.customer_id = B.customer_id AND A.started_at <= B.converted_at
        
        WHERE B.customer_id IS NOT NULL
        GROUP BY B.customer_id ;
'''

pd.read_sql_query(q, conn)

,customer_id,num_of_touchpoints
0,1170,3
1,2956,3


### Q3)* Bonus if you can also add ad spend data & show ROI at the end

In [25]:
# For example - in sessions table we have a column for marketing_cost that indicates the money spent on that specific marketing
# Conversions table has a column subscription_cost that is paid by the customer after conversion

# we can use the above data to calcualte ROI for a specific ad from below formula (Assuming organic growth = 0 for simplicity)
# ROI = (Sales Growth - Average Organic Sales Growth - Marketing Cost) / Marketing Cost
# query would be similar to ques1 (part 2) - we just have to SUM the marketing cost and customer_subscription_cost and
# use the ROI formula

In [26]:
conn.close()